# InfVis project work
---
This Python Script downloads an up-to-date dataset for **Covid-19**, gives first insights on the data and exports the data to the data folder.

The Data is from the **R**obert **K**och **I**nstitut downloaded over [ArcGis Hub](https://hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6?page=15976).

*Script was created on Python: 3.7.6 64-bit Kernel*

## Importing and installing of required packages

In [68]:
%matplotlib inline
import pandas as pd
import numpy as np
import math

import io               # file operations
import json

import ssl              # secure client-server connection
import requests         # html-requests

In [111]:
# Uncomment next 2 lines to install jsonmerge
#import sys
#!{sys.executable} -m pip install jsonmerge
from jsonmerge import Merger

## Pulling the data from the internet

### Defining URL´s
URL´s are created from stringconcat.

In [44]:
sourceURL = 'https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query?'
objectIdsQuery = 'where=1%3D1&returnIdsOnly=true&f=json'
dataSetQuery = 'where=ObjectId+BETWEEN+0+AND+0' # just as an example gets created later dynamically
dataQuery = '&outSR=4326&outFields=IdBundesland,Bundesland,Landkreis,Altersgruppe,AnzahlFall,AnzahlTodesfall,ObjectId,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn&f=json'

In [108]:
jsonMergeSchema = {"properties":{"features":{"mergeStrategy":"append"}}}
dataMerger = Merger(jsonMergeSchema)

### First request
Identifies downloadable data.

In [36]:
objectIdsRequest = requests.get(sourceURL + objectIdsQuery)
objectIdsRequest.status_code

200


In [53]:
objectIds = json.loads(objectIdsRequest.text)

numOfObjectIds = len(objectIds['objectIds'])

objectIdStart = objectIds['objectIds'][0]
objectIdEnd = objectIds['objectIds'][numOfObjectIds - 1]
print(f'Range of ObjectIds: [{objectIdStart}, {objectIdEnd}]')

Number of ObjectIds / Datasets: 159759
The download will require 32 requests due to the server limit of 5000 features/request.
Range of ObjectIds: [17147720, 17307478]


### Secound request
Determines eventual limit of data transfered by the server

In [75]:
dataRequest = requests.get(sourceURL + 'where=1%3D1' + dataQuery)
dataRequest.status_code

200

In [122]:
data = json.loads(dataRequest.text)
maxApiRequest = len(data['features'])

neededRequests = math.ceil(numOfObjectIds / maxApiRequest)

print(f'The download will require {neededRequests - 1} more requests due to the server limit of {maxApiRequest} features/request.')

The download will require 31 more requests due to the server limit of 5000 features/request.


Create all requests needed to get the data for every single ObjectId.  
**Ensure the cell above is run before this one is!**  
Otherwise it **WILL** cause data to be duplicated

In [123]:
i = 0
rangeLowerEnd = data['features'][maxApiRequest - 1]['attributes']['ObjectId'] + 1
rangeUpperEnd = rangeLowerEnd + maxApiRequest

while (i < neededRequests - 1): # neededRequests - 1 because of initial download
    dataSetQuery = f'where=ObjectId+BETWEEN+{rangeLowerEnd}+AND+{rangeUpperEnd}'
    temp_sourceURL = sourceURL + dataSetQuery + dataQuery
    #print(i, temp_sourceURL)
    print(i, f'Pulling ObjectIds: [{rangeLowerEnd}, {rangeUpperEnd}]')

    temp_dataRequest = requests.get(temp_sourceURL)
    if (temp_dataRequest.status_code > 200): # stop when a request isn´t working
        print(temp_dataRequest.status_code)
        break

    temp_data = json.loads(temp_dataRequest.text)

    data = dataMerger.merge(data, temp_data)

    temp_dataLength = len(data['features'])
    t_le = data['features'][0]['attributes']['ObjectId']
    t_ue = data['features'][temp_dataLength - 1]['attributes']['ObjectId']
    print(f'Total collected features: {temp_dataLength}', f'From ObjectIds: [{t_le}, {t_ue}]')

    rangeLowerEnd = rangeUpperEnd + 1
    rangeUpperEnd += maxApiRequest + 1
    if (rangeUpperEnd > objectIdEnd):
        rangeUpperEnd = objectIdEnd
    i += 1

print('Done')

0 Pulling ObjectIds: [17152720, 17157720]
Total collected features: 10000 From ObjectIds: [17147720, 17157719]
1 Pulling ObjectIds: [17157721, 17162721]
Total collected features: 15000 From ObjectIds: [17147720, 17162720]
2 Pulling ObjectIds: [17162722, 17167722]
Total collected features: 20000 From ObjectIds: [17147720, 17167721]
3 Pulling ObjectIds: [17167723, 17172723]
Total collected features: 25000 From ObjectIds: [17147720, 17172722]
4 Pulling ObjectIds: [17172724, 17177724]
Total collected features: 30000 From ObjectIds: [17147720, 17177723]
5 Pulling ObjectIds: [17177725, 17182725]
Total collected features: 35000 From ObjectIds: [17147720, 17182724]
6 Pulling ObjectIds: [17182726, 17187726]
Total collected features: 40000 From ObjectIds: [17147720, 17187725]
7 Pulling ObjectIds: [17187727, 17192727]
Total collected features: 45000 From ObjectIds: [17147720, 17192726]
8 Pulling ObjectIds: [17192728, 17197728]
Total collected features: 50000 From ObjectIds: [17147720, 17197727]
9

In [116]:
for item in data['fields']:
    print(item['name'])

IdBundesland
Bundesland
Landkreis
Altersgruppe
AnzahlFall
AnzahlTodesfall
ObjectId
Meldedatum
IdLandkreis
Datenstand
NeuerFall
NeuerTodesfall
Refdatum
NeuGenesen
AnzahlGenesen
IstErkrankungsbeginn


Maximum number of features per query

In [117]:
len(data['features'])

159729

In [118]:
totalCases = 0
for entry in data['features']:
    totalCases += entry['attributes']['AnzahlFall']
totalCases

198093

Testing commulated data for comparison with numbers from the internet

In [124]:
shDeaths = 0
shRecovered = 0
shEntries = 0
for entry in data['features']:
    if entry['attributes']['Bundesland'] == 'Schleswig-Holstein':
        shDeaths += entry['attributes']['AnzahlTodesfall']
        shRecovered += entry['attributes']['AnzahlGenesen']
        shEntries += 1
print('Deaths:', shDeaths, 'Recovered:', shRecovered, 'Entries:', shEntries)

Deaths: 154 Recovered: 3011 Entries: 2884
